In [2]:
import pandas as pd
import numpy as np

In [110]:
historic_orig = pd.read_csv('data/NYPD_Shooting_Incident_Data__Historic_.csv')
ytd_orig = pd.read_csv('data/NYPD_Shooting_Incident_Data__Year_To_Date_.csv')

In [111]:
historic_orig.columns

Index(['INCIDENT_KEY', 'OCCUR_DATE', 'OCCUR_TIME', 'BORO', 'PRECINCT',
       'JURISDICTION_CODE', 'LOCATION_DESC', 'STATISTICAL_MURDER_FLAG',
       'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'VIC_AGE_GROUP', 'VIC_SEX',
       'VIC_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude'],
      dtype='object')

In [112]:
def clean(df):
    df['DATESTAMP'] = pd.to_datetime(df.OCCUR_DATE + ' ' + df.OCCUR_TIME, errors='coerce')
    df['LOCATION_CODE'] = str(df['BORO']) + ' ' + str(df['PRECINCT']) + ' ' + str(df['JURISDICTION_CODE'])
    return df.drop(['OCCUR_DATE', 'OCCUR_TIME', 'X_COORD_CD', 'Y_COORD_CD', 'BORO', 'PRECINCT', 'JURISDICTION_CODE'], axis=1)

historic_clean = clean(historic_orig)
ytd_clean = clean(ytd_orig)

In [113]:
main = pd.concat([historic_clean, ytd_clean])

In [122]:
main.describe(include=[np.object])

,LOCATION_DESC,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,LOCATION_CODE
count,2344,3635,3635,3635,6848,6848,6848,6848
unique,27,8,3,7,6,3,7,2
top,MULTI DWELL - PUBLIC HOUS,18-24,M,BLACK,25-44,M,BLACK,0 BROOKLYN\n1 BROOKLYN\n...
freq,1208,1582,3514,2596,3064,6224,4924,6407


In [123]:
main.sample(1)

,INCIDENT_KEY,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,Latitude,Longitude,DATESTAMP,LOCATION_CODE
4781,109469979,NaN,False,<18,M,BLACK,<18,M,BLACK,40.662487,-73.904012,2014-04-07 18:19:00,0 BROOKLYN\n1 BROOKLYN\n...


In [25]:
import folium
from IPython.display import HTML

In [28]:
# https://app.dominodatalab.com/u/r00sj3/crimemaps/view/examples.ipynb
def display(m, height=300):
    """Takes a folium instance and embed HTML."""
    srcdoc = m.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{0}" '
                 'style="width: 100%; height: {1}px; '
                 'border: none"></iframe>'.format(srcdoc, height))
    return embed

In [31]:
print(ytd_orig.columns)

Index(['INCIDENT_KEY', 'OCCUR_DATE', 'OCCUR_TIME', 'BORO', 'PRECINCT',
       'JURISDICTION_CODE', 'LOCATION_DESC', 'STATISTICAL_MURDER_FLAG',
       'PERP_AGE_GROUP', 'PERP_SEX', 'PERP_RACE', 'VIC_AGE_GROUP', 'VIC_SEX',
       'VIC_RACE', 'X_COORD_CD', 'Y_COORD_CD', 'Latitude', 'Longitude'],
      dtype='object')


In [ ]:
# EDA: how many types/value_counts of BORO, PRECINT, LOCATION

# Missing Data Analysis

# Time trends plots
# Groupby month/year, count #shootings
# Groupby month/year, count #race (perp/victim)
# Groupby month/year, count #age_group (perp/victim)
# Groupby month/year, count #sex (perp/victim)
# Groupby month/year, count #age_group (perp/victim)

# Visualization flag for police shootings, flag for Statistical Murder Flag
# Murder/non-murder, male, female

In [54]:
historic_orig.sample(1)

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude,datestamp
1114,158273233,11/04/2016 12:00:00 AM,14:45:00,BRONX,43,0,NaN,True,25-44,M,WHITE HISPANIC,UNKNOWN,M,UNKNOWN,1020999,244291,40.837129,-73.867195,2016-11-04 14:45:00


In [74]:
ytd_orig.sample(1)

,INCIDENT_KEY,OCCUR_DATE,OCCUR_TIME,BORO,PRECINCT,JURISDICTION_CODE,LOCATION_DESC,STATISTICAL_MURDER_FLAG,PERP_AGE_GROUP,PERP_SEX,PERP_RACE,VIC_AGE_GROUP,VIC_SEX,VIC_RACE,X_COORD_CD,Y_COORD_CD,Latitude,Longitude
77,183609561,06/07/2018 12:00:00 AM,17:14:00,BRONX,44,0,NaN,False,NaN,NaN,NaN,18-24,M,WHITE HISPANIC,1004689,243518,40.835061,-73.92614


In [94]:
import datetime as dt

In [99]:
def create_folium_map(df):
    NYC_COORDINATES = (40.730610, -73.935242)
    MORNING_URL = 'http://icons.iconarchive.com/icons/icons-land/weather/256/Sunrise-icon.png'
    NIGHT_URL = 'http://cdn.onlinewebfonts.com/svg/download_169174.png'
    morning_icon = folium.features.CustomIcon(MORNING_URL, icon_size=(28, 30))
    night_icon = folium.features.CustomIcon(NIGHT_URL, icon_size=(28, 30))
    m = folium.Map(location=NYC_COORDINATES, zoom_start=10)
    for i, row in df.sample(MAX_RECORDS).iterrows():
        icon_color = 'blue'
        custom_icon = morning_icon
        if row.STATISTICAL_MURDER_FLAG: icon_color = 'red'
        print(type(row.datestamp))
        if row.datestamp > dt.time('12:00'): custom_icon = night_icon
        folium.Marker(location=[row.Latitude, row.Longitude], popup=str(row.datestamp), 
                      icon=icon).add_to(m)
    return m

In [100]:
create_folium_map(clean(ytd_orig))

<class 'pandas._libs.tslib.Timestamp'>


TypeError: an integer is required (got type str)